# Using Altair

Altair is a very sophisticated package for visualizing data. It may be overkill for simple dataviz (and `seaborn` would suffice), but it is very useful when we need to zoom in and out on the data, select elements, etc. Think of it as a dynamic version of `seaborn`.

Let's see how it works. Let's load the environment and then use a specific dataset.

In [ ]:
# we don't like warnings
# you can comment the following 2 lines if you'd like to
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import altair as alt

## Load the dataset

This time we will use a different dataset. It contains information about cars. The dataset is available for free, you just need to install the `vega_datasets` package that is provided by Altair.

In [ ]:
from vega_datasets import data

# get the dataset
cars = data.cars.url

## Altair

Ok, time to do fancy stuff. Let's plot a dynamic scatter and bar charts that interact with one another. When you click on one, it filters the values of the other.

Let's start by creating an interval selection, which allows for brushing (click and drag) over points in a chart

In [ ]:
brush = alt.selection_interval()

Now let's create a scatter plot (points) using the cars dataset.

It shows the relationship between `Horsepower` and `Miles_per_Gallon` and colors the points by their `Origin`

In [ ]:
points = alt.Chart(cars).mark_point().encode(
    # Quantitative encoding for x-axis: Horsepower
    x='Horsepower:Q',
    # Quantitative encoding for y-axis: Miles_per_Gallon
    y='Miles_per_Gallon:Q',
    # Nominal encoding for color: Origin
    color='Origin:N'
).add_params(
    # Add the interval selection (brushing) functionality to the scatter plot
    brush
)

Create a bar chart (bars) using the cars dataset. It counts the number of cars from each origin.

In [ ]:
bars = alt.Chart(cars).mark_bar().encode(
    # Count the number of records for the x-axis
    x='count()',
    # Nominal encoding for y-axis: Origin
    y='Origin:N',
    # Nominal encoding for color: Origin
    color='Origin:N'
).transform_filter(
    # Filter the bars based on the brushed selection in the scatter plot
    brush
)


Finally, display the scatter plot (points) and the bar chart (bars) together (vertically stacked).

In [ ]:
points & bars

Good job! Let's now draw another chart to continue exploring the potential of Altair. We will create an interactive scatter plot where the user can move a slider to adjust a cutoff value. Points to the left of this cutoff are colored red, and points to the right are colored blue. The slider allows the user to dynamically change this cutoff and observe the resulting color changes in the plot.

The following block of code explains along the way what is being done.

In [ ]:
# Seed the random generator for reproducibility
rand = np.random.RandomState(42)

# Create a DataFrame with two columns: 'xval' and 'yval'
# 'xval' is a range from 0 to 99
# 'yval' is a cumulative sum of 100 random numbers drawn from a normal distribution
df = pd.DataFrame({
    'xval': range(100),
    'yval': rand.randn(100).cumsum()
})

# Create a slider UI element for user interaction
# This slider ranges from 0 to 100 and will be used to set a cutoff value
slider = alt.binding_range(min=0, max=100, step=1, name='Cutoff ')

# Define a parameter that will be bound to the slider UI
# The parameter is initialized with a value of 50
selector = alt.param(name='SelectorName', value=50, bind=slider)

# Create a scatter plot using the DataFrame 'df'
# Points to the left of the slider value will be colored red, and to the right will be colored blue
alt.Chart(df).mark_point().encode(
    # Encode x-axis with 'xval' values
    x='xval',
    # Encode y-axis with 'yval' values
    y='yval',
    color=alt.condition(
        # Condition to check if 'xval' is less than the slider value
        alt.datum.xval < selector,
        # An equivalent alternative for the condition
        # 'datum.xval < SelectorName',
        # Color to use when condition is true
        alt.value('red'),
        # Color to use when condition is false
        alt.value('blue')
    )
).add_params(
    selector  # Attach the slider UI to the chart for user interaction
)